In [ ]:
from pyspark.sql.functions import from_unixtime, col, round, to_date

In [ ]:
df = spark.read.format('delta').load("dbfs:/FileStore/tables/bronze/user_details")

In [ ]:
# Converte para timestamp
df = df.withColumn(
    'rtime_last_played',
    from_unixtime(col('rtime_last_played')).cast('timestamp')
)

# Conversão minutos → horas com 2 casas decimais
df = df.withColumn(
    'playtime_forever',
    round(col('playtime_forever') / 60, 2).cast('double')
)

# Filtrar apenas valores maiores que 0.00
df = df.filter(col('playtime_forever') > 0.00)

# Converte de string para date
df = df.withColumn("data_ref", to_date(col("data_ref"), "yyyy-MM-dd"))

df = df.select('appid','playtime_forever','rtime_last_played','data_ref')

In [ ]:
df.write.format("delta") \
    .mode("overwrite") \
    .partitionBy("data_ref") \
    .save("dbfs:/FileStore/tables/silver/user_details")